# Summary

**Objective**

We have a sensor data source in JSON format from our IOT devices. We would like to create an analytics data lake and allow analyst to query the data.

**Challenge**

The raw data are complex JSON, we would like to flatten it out into parquet.

# Step 1. Preparation

## Step 1.1 Import Libraries

In [ ]:
# Standard Library
import os
import sys
import dataclasses

# PySpark
from pyspark.sql import SparkSession

# Glue
from pyspark.context import SparkContext
from awsglue import DynamicFrame
from awsglue.context import GlueContext
from awsglue.utils import getResolvedOptions
from awsglue.job import Job

spark_ses = SparkSession.builder.getOrCreate()
# spark_ctx = SparkContext.getOrCreate()
# glue_ctx = GlueContext(spark_ctx)
# spark_ses = glue_ctx.spark_session

In [ ]:
pdf = spark_ses.createDataFrame(
    [
        ("e-1", "2000-01-01T08:30:00", {"temperature": 78}),
        ("e-2", "2000-01-01T08:30:00", {"temperature": 56}),
        ("e-3", "2000-01-01T08:30:00", {"temperature": 69}),
    ],
    ("device_id", "time", "measurement"),
)
pdf.show()

## Step 1.2 Detect the Current Runtime

In development phase, we would like to use interactive Jupyter Notebook and small sample data as the source. So we can focus on the transformation logics and see live results faster.

In [4]:
IS_CONTAINER = False
IS_GLUE_NOTEBOOK = False
IS_GLUE_JOB = False

# The Glue Job run always has "--JOB_RUN_ID" in arguments
if "--JOB_RUN_ID" in sys.argv:
    IS_GLUE_JOB = True
# We always pass in a custom env var in "docker run ..." command
elif os.environ.get("IS_CONTAINER", "n") == "y":
    IS_CONTAINER = True
else:
    IS_GLUE_NOTEBOOK = True


def print_header(msg: str):
    msg = f" {msg} "
    bar = "=" * 10
    print(f"{bar}{msg:=<70}")


print_header("Step 1. Preparation")
print("Runtime Information:")
print("")
print(f"IS_CONTAINER = {IS_CONTAINER}")
print(f"IS_GLUE_NOTEBOOK = {IS_GLUE_NOTEBOOK}")
print(f"IS_GLUE_JOB = {IS_GLUE_JOB}")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

========== Runtime Information =================================================
IS_CONTAINER = True
IS_GLUE_NOTEBOOK = False
IS_GLUE_JOB = False

In [7]:
# Import custom libraries
if IS_CONTAINER:
    more_path = [
        "/home/glue_user/workspace/jupyter_workspace",
        "/home/glue_user/workspace/extra_python_path",
    ]
    for path in more_path:
        if path not in sys.path:
            sys.path.append(path)

from gluelib_json_to_parquet.transform_dataframe import json_to_parquet

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [23]:
# Initialize Job (if in glue job runtime)
if IS_GLUE_JOB:
    args = getResolvedOptions(
        sys.argv,
        [
            "JOB_NAME",
            "data_source_s3_uri",
        ]
    )
    print("Resolved Arguments:")
    print("")
    for key, value in args.items():
        print(f"{key} = {value!r}")
    job = Job(glue_ctx)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

## Step 1.3 Declare Parameter

We declared a data class to store all required Job Parameters.

In production, most of parameter should be passed in as [Job Arguments](https://docs.aws.amazon.com/glue/latest/dg/aws-glue-api-crawler-pyspark-extensions-get-resolved-options.html).

In development phase, we would like to hardcode the value so we can focus on the core logics.

We could use a simple ``if else`` statement to use different logic to initialize the Job Parameters Python object.

In [9]:
@dataclasses.dataclass
class Param:
    data_source_s3_uri: str = ""

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

## Step 1.4 Resolve Parameter

In [10]:
if IS_CONTAINER or IS_GLUE_NOTEBOOK:
    param = Param(
        data_source_s3_uri="s3://501105007192-us-east-1-data/projects/gluelib_json_to_parquet/data/sensors/"
    )
else:
    param = Param(
        data_source_s3_uri=args["data_source_s3_uri"],
    )

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

# Step 2. Read The Data

In [19]:
print_header("Step 2. Read the data")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

========== Step 2. Read the data ===============================================

## Step 2.1 Read Sensor Data

In most of ETL job, we need to read all required data into dataframe. In this section, we focus on reading the data from different data source.

**Use Small Data in Dev**

The Glue Job is designed for big data processing. However in ETL development phase, the most creative part is the data wrangling. Debugging with big data is not only very expansive and also very NON-interactive. In development phase, I recommend to use small data with similar data characters for development.

In [11]:
def fake_sensor_data() -> DynamicFrame:
    """
    Fake the sensor data source for dev.
    """
    pdf = spark_ses.createDataFrame(
        [
            ("e-1", "2000-01-01T08:30:00", {"temperature": 78}),
            ("e-2", "2000-01-01T08:30:00", {"temperature": 56}),
            ("e-3", "2000-01-01T08:30:00", {"temperature": 69}),
        ],
        ("device_id", "time", "measurement"),
    )
    return DynamicFrame.fromDF(pdf, glue_ctx, "gdf_sensor")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [12]:
def read_sensor_data_from_s3() -> DynamicFrame:
    """
    Read data from AWS S3 bucket
    """
    return glue_ctx.create_dynamic_frame.from_options(
        connection_type="s3",
        connection_options=dict(
            paths=[
                param.data_source_s3_uri,
            ],
            recurse=True
        ),
        format="json",
        format_options=dict(multiLine="true"),
        transformation_ctx="gdf",
    )

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [13]:
if IS_CONTAINER or IS_GLUE_NOTEBOOK:
    gdf_sensor = fake_sensor_data()
else:
    gdf_sensor = read_sensor_data_from_s3()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [14]:
gdf_sensor.toDF().show(3)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+---------+-------------------+-------------------+
|device_id|               time|        measurement|
+---------+-------------------+-------------------+
|      e-1|2000-01-01T08:30:00|{temperature -> 78}|
|      e-2|2000-01-01T08:30:00|{temperature -> 56}|
|      e-3|2000-01-01T08:30:00|{temperature -> 69}|
+---------+-------------------+-------------------+

# Step 3. Transform The Data

In this section, we focus on transform the data into desired schema, format.

In [20]:
print_header("Step 3. Transform the data")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

========== Step 3. Transform the data ==========================================

In [15]:
gdf_sensor_unnested = json_to_parquet(gdf_sensor, "gdf_sensor")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [16]:
gdf_sensor_unnested.toDF().show(3)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+---------+-------------------+-----------------------+
|device_id|               time|measurement.temperature|
+---------+-------------------+-----------------------+
|      e-1|2000-01-01T08:30:00|                     78|
|      e-2|2000-01-01T08:30:00|                     56|
|      e-3|2000-01-01T08:30:00|                     69|
+---------+-------------------+-----------------------+

# Step 4. Write the Data

In [21]:
print_header("Step 4. Write the data")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

========== Step 4. Write the data ==============================================

In [17]:
# dump your data to desired location in desired format

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

# Step 5. Clean Up

In [22]:
print_header("Step 5. Clean Up")
if IS_GLUE_JOB:
    job.commit()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

========== Step 5. Clean Up ====================================================